In [75]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import sys

In [2]:
metadata_df = pd.read_json("../data/metadata_rich_df.json")


In [51]:
sourcepath = "../data/large_files/trigrams_lemmatized/"
with open(sourcepath + "trigrams_lemmata_id_0.txt".format(str(id)), "r") as f:
    trigrams_list = f.readlines()

In [52]:
[el.strip() for el in trigrams_list][:10]

['religious matter',
 'divine hope',
 'divine hope',
 'leave unresolved',
 'leave unresolved',
 'racial equality',
 'racial equality',
 'joint action',
 'joint action',
 'joint action']

# Load vocabulary

In [53]:
bidecades_vocabs_counts_df = pickle.load(open("../data/bidecades_vocabs_counts_df.pickle", "rb"))
bidecades_vocabs_counts_df.head(5)

,1900-1919,1920-1939,1940-1959,1960-1979,1980-1999,2000-2019
man,11218.0,8342.0,24108.0,39199.0,48435.0,100670.0
new,10183.0,10192.0,23469.0,42092.0,97421.0,293457.0
Jesus,9650.0,4911.0,12134.0,23416.0,49765.0,215687.0
et,9325.0,11818.0,20805.0,35969.0,49326.0,73464.0
God,9248.0,6276.0,26328.0,45751.0,99505.0,293240.0


In [54]:
vocabulary = dict([(el[1], el[0]) for el in enumerate(bidecades_vocabs_counts_df.index)])

# Load trigrams

In [116]:
bidecades_strs = sorted([el for el in list(set(metadata_df["bidecade"])) if el != None])
bidecades_strs

['1900-1919', '1920-1939', '1940-1959', '1960-1979', '1980-1999', '2000-2019']

In [117]:
bidecade = "1900-1919"
ids = metadata_df[metadata_df["bidecade"]==bidecade]["id_kase"]
len(ids)

782

In [118]:
with open("../data/large_files/bidecade_trigrams_{}.txt".format(bidecade), "r") as f:
    subcorpus_ngrams = f.readlines()

In [119]:
[el.strip() for el in subcorpus_ngrams[:10]]

['Michigan Ann Arbor',
 'Bonner University',
 'Bonner University',
 'Iowa Iowa',
 'Iowa Iowa',
 'Kansas Lawrence',
 'Ginn Co',
 'Elliott University',
 'Elliott University',
 'Kansas Lawrence Kansas']

In [120]:
vectorizer = TfidfVectorizer(vocabulary=vocabulary, lowercase=False)
X = vectorizer.fit_transform(subcorpus_ngrams)

In [121]:
X.shape

(722798, 4804)

In [122]:
cooc = X.T * X

In [123]:
cooc.max()

6277.248125246523

In [124]:
cooc.todense().max()

6277.248125246523

In [125]:
cooc_norm_dict = {}
for bidecade in bidecades_strs:
    with open("../data/large_files/bidecade_trigrams_{}.txt".format(bidecade), "r") as f:
        subcorpus_ngrams = f.readlines()
    vectorizer = TfidfVectorizer(vocabulary=vocabulary, lowercase=False)
    X = vectorizer.fit_transform(subcorpus_ngrams)
    cooc = X.T * X
    cooc_norm_dict[bidecade] = cooc

In [127]:
pickle.dump(cooc_norm_dict, open("../data/cooc_dict.pickle", "wb"))

In [126]:
cooc_norm_dict["1900-1919"]["Paul"].sort_values(ascending=False)

IndexError: Index dimension must be 1 or 2

In [39]:
cooc_norm_dict["1980-1999"]["Paul"].sort_values(ascending=False)

Paul        1.000000
letter      0.052684
Apostle     0.025385
use         0.021093
epistle     0.016693
              ...   
opposite    0.000000
Simeon      0.000000
withdraw    0.000000
vote        0.000000
less        0.000000
Name: Paul, Length: 4804, dtype: float64